Unfinished - changed approach. See - firstpassb notebook.

# Reading in data

In [55]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import re 
from pandas_profiling import ProfileReport

import matplotlib.pyplot as plt

In [56]:
path = 'alldata/'


In [57]:
def make_dataframes(path):
    file_names = [pos_csv for pos_csv in os.listdir(path) if pos_csv.endswith('.csv')]
    dataframes = {}
    dfs = []
    for i, file in enumerate(file_names):
        df_name = ("df_" + file)[:-4]
        dfs.append(pd.read_csv(path + file))
        dataframes[df_name] = dfs[i]
    return dataframes, file_names

data_dict, files = make_dataframes(path)

In [58]:
# We created a dictionary of dataframes:
data_dict.keys()
# These are the dataframes:

dict_keys(['df_current_log', 'df_weather_log', 'df_weekpop_df', 'df_binned_weather_history', 'df_binned_current_log', 'df_image_label', 'df_ProspectParkSurvey', 'df_curpop_df', 'df_image_label_isa', 'df_Survey_clean'])

# Pre-merge EDA

In [59]:
# We join them on the time_bin column:
data_dict['df_image_label_isa'] = data_dict['df_image_label_isa'].rename(columns={"timetaken": "time_bin"})

In [60]:
def fix_time_format(df):
    """ 
    Turn format to same format for df with 'time_bin' datetime object column.
    """
    print("before:", df['time_bin'].dtype)
    df['time_bin']= pd.to_datetime(df['time_bin']) 
    print("after:", df['time_bin'].dtype)
#    df['time_bin'] = df['time_bin'].dt.strftime('%Y-%m-%d %H:%M')


In [61]:
# Read in pkl tweets into dataframe:
path_to_pkl = "alldata/geotweets_labeled_binned.pkl"

def read_pkl(path_to_pkl):
    tweets = pd.read_pickle(path_to_pkl)
    
    # Create tweets_safe (time bin column, safe score column)
    print(tweets.dtypes)
   # tweets['time_bin'] = tweets['time_bin'].astype(str)
    tweets_safe = tweets[pd.notnull(tweets['safe'])]
#     tweets_safe = tweets.groupby('time_bin').agg(avg_safe=('safe', 'mean'), num_tweets =('time_bin', 'count'))
#     #tweets_safe = tweets.groupby('time_bin').agg({'safe':['mean'], 'safe':['sum']})
#     print(len(tweets_safe))
#     tweets_safe = tweets_safe.drop(columns=['num_tweets'])
#     tweets_safe.columns
    tweets_safe = tweets_safe.reset_index()
    
    # tweets need to be changed to stardard labels: safe = 0, unsafe = 1
    tweets_safe["safe"] = tweets_safe["safe"].replace([0, 1], [1,0])
    
    #fix_time_format(tweets_safe)

    
    # create tweets_count (time bin column, number of tweets)
    tweets_count = tweets.groupby('time_bin').agg(avg_safe=('safe', 'mean'), num_tweets =('time_bin', 'count'))
    tweets_count = tweets_count.drop(columns=['avg_safe'])
    tweets_count = tweets_count.reset_index()
    
    
    return tweets_safe, tweets_count

tweets_safe, tweets_count = read_pkl(path_to_pkl)

date           datetime64[ns, US/Eastern]
text                               object
username                           object
day_of_week                        object
hour                                int64
safe                              float64
time_bin                   datetime64[ns]
dtype: object


In [62]:
considered_dfs = ['df_curpop_df',
                  'df_binned_current_log',
                  'df_image_label_isa',
                  'df_Survey_clean',
                  'df_binned_weather_history']

def convert_and_examine_time(considered_dfs):
    
    for df in considered_dfs:
        print("df name:", df)
        fix_time_format(data_dict[df])
        print('begins:', data_dict[df]['time_bin'].min(), 
              "\nends:", data_dict[df]['time_bin'].max(), 
              "\nnon-na vals:", data_dict[df]['time_bin'].count(), "\n\n" )
    
    print("df name:", "tweets_safe")
    print('begins:', tweets_safe['time_bin'].min(), 
          "\nends:", tweets_safe['time_bin'].max(), 
          "\nnon-na vals:", tweets_safe['time_bin'].count())

# fix_time_format(data_dict['df_curpop_df'])
# fix_time_format(data_dict['df_binned_current_log'])
# fix_time_format(data_dict['df_image_label_isa'])
# fix_time_format(data_dict['df_Survey_clean'])
# fix_time_format(data_dict['df_binned_weather_history'])

convert_and_examine_time(considered_dfs)

df name: df_curpop_df
before: object
after: datetime64[ns]
begins: 2020-04-02 15:05:00 
ends: 2020-04-09 17:05:00 
non-na vals: 663 


df name: df_binned_current_log
before: object
after: datetime64[ns]
begins: 2020-04-04 08:35:00 
ends: 2020-04-10 15:50:00 
non-na vals: 483 


df name: df_image_label_isa
before: object
after: datetime64[ns]
begins: 2020-04-02 16:05:00 
ends: 2020-04-07 18:20:00 
non-na vals: 54 


df name: df_Survey_clean
before: object
after: datetime64[ns]
begins: 2020-03-29 10:50:00 
ends: 2020-04-11 16:20:00 
non-na vals: 22 


df name: df_binned_weather_history
before: object
after: datetime64[ns]
begins: 2019-12-31 23:50:00 
ends: 2020-04-11 22:50:00 
non-na vals: 9789 


df name: tweets_safe
begins: 2020-03-23 14:05:00 
ends: 2020-04-13 07:05:00 
non-na vals: 32


# Make one big dataframe

In [63]:
# Join dataframes on time_bin column:

df_full = pd.merge(data_dict['df_curpop_df'], data_dict['df_binned_weather_history'], on='time_bin', how='outer')
df_full = pd.merge(df_full, data_dict['df_image_label_isa'], on='time_bin', how='outer')
df_full = pd.merge(df_full, data_dict['df_image_label_isa'], on='time_bin', how='outer')
df_full = pd.merge(df_full, data_dict['df_binned_current_log'], on='time_bin', how='outer')

df_full = pd.merge(df_full, tweets_safe, on='time_bin', how='outer')
df_full = pd.merge(df_full, tweets_count, on='time_bin', how='outer')


In [64]:
df_full.head(1)

,Unnamed: 0_x,current_popularity,datetime,time_bin,time,temperature,dewpoint,humidity_x,precipitation,windspeed,...,status,detailed_status,index,date,text,username,day_of_week,hour,safe,num_tweets
0,0.0,38.0,4/2/2020 15:17,2020-04-02 15:05:00,NaN,48.785,-0.65,49.75,0.0,44.075,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN


# Post-merge EDA

In [21]:
# Pandas profiling Report
profile = ProfileReport(df_full, title='report', html={'style':{'full_width':True}})
profile.to_file(output_file="report.html")

In [65]:
# We can see we are missing a lot of data for some variables:
df_full.count()

Unnamed: 0_x            663
current_popularity      663
datetime                663
time_bin              10029
time                   2508
temperature           10007
dewpoint              10007
humidity_x            10007
precipitation         10007
windspeed             10007
winddirection         10007
pressure              10007
Unnamed: 0_y             82
label_x                  84
Isa_x                    84
Unnamed: 0               82
label_y                  84
Isa_y                    84
clouds                  612
rain_1h                 612
snow_1h                 612
wind_speed              612
wind_deg                612
humidity_y              612
press                   612
temp                    612
temp_feels              612
temp_max                612
temp_min                612
park_name               612
sunrise_time            612
sunset_time             612
status                  612
detailed_status         612
index                    41
date                

In [66]:
df_full.shape

(10029, 42)

Project criteria: **We will only include data from March 23rd, 2020 onward**:

In [67]:
df = df_full[df_full['time_bin'] >= '3/23/2020 00:00']
print("before filtering:", df_full.shape, "\nafter filtering:", df.shape)

before filtering: (10029, 42) 
after filtering: (2156, 42)


# Building a target variable, and some feature engineering

We have a few variables we can use to build a labelled column - we have a lot of missing values, so we can cobble a target variable using different columns:

- main_track: from survey data on how busy the main track of the park is (df_Survey_clean)
- label, Isa: from photos (df_image_label_isa)
- tweets_safe: from twitter scraping data


In [ ]:
target_var = ['main_track', 'label', 'Isa', 'tweets_safe']
indep_vars = ['activity', 'current_popularity']

In [43]:
print(df['time_bin'].nunique())
print(len(df))

1938
2155


Naively drop any target variables that we don't have zeros or ones for:

In [26]:
data_dict.keys()
temperature, precipitation, windspeed, detailed_status, current_popularity

dict_keys(['df_current_log', 'df_weather_log', 'df_weekpop_df', 'df_binned_weather_history', 'df_binned_current_log', 'df_image_label', 'df_ProspectParkSurvey', 'df_curpop_df', 'df_image_label_isa', 'df_Survey_clean'])

In [41]:
data_dict['df_binned_weather_history']

,time_bin,time,temperature,dewpoint,humidity,precipitation,windspeed,winddirection,pressure
0,2019-12-31 23:50:00,2020-01-01 00:00:00,42.080,3.900,89.00,0.0,13.00,220.0,1004.10
1,2020-01-01 00:05:00,NaN,42.305,3.900,88.25,0.0,13.90,217.5,1004.00
2,2020-01-01 00:20:00,NaN,42.530,3.900,87.50,0.0,14.80,215.0,1003.90
3,2020-01-01 00:35:00,NaN,42.755,3.900,86.75,0.0,15.70,212.5,1003.80
4,2020-01-01 00:50:00,2020-01-01 01:00:00,42.980,3.900,86.00,0.0,16.60,210.0,1003.70
...,...,...,...,...,...,...,...,...,...
9784,2020-04-11 21:50:00,2020-04-11 22:00:00,53.960,-4.900,30.00,0.0,31.70,270.0,1013.10
9785,2020-04-11 22:05:00,NaN,53.735,-5.225,29.50,0.0,31.15,272.5,1013.25
9786,2020-04-11 22:20:00,NaN,53.510,-5.550,29.00,0.0,30.60,275.0,1013.40
9787,2020-04-11 22:35:00,NaN,53.285,-5.875,28.50,0.0,30.05,277.5,1013.55
